# Simplified `Compressor`

We consider a component made of aerodynamics and geometry.

In [ ]:
from pyturbo.systems.compressor_simple import CompressorSimple
from cosapp.drivers import NonLinearSolver, RunSingleCase

cmp = CompressorSimple('cmp', stage_count = 1)
run = cmp.add_driver(NonLinearSolver('run'))

## simulation

As for isolated aero, we could release `fl_in.W` or `shaft_in.power`. 

In [ ]:
run.add_unknown('shaft_in.power')

In [ ]:
cmp.geom.scale = 1.1
cmp.fl_in.W = 400.
cmp.shaft_in.N = 5000.

cmp.run_drivers()

print('mass flow : ', cmp.fl_in.W,' kg/s')
print('shaft power : ', cmp.shaft_in.power / 1e6, ' MW')
print('shaft N : ', cmp.shaft_in.N, ' rpm')
print('pt_ratio : ', cmp.aero.pr)
print('Tt_ratio : ', cmp.aero.tr)

### calibration

In [ ]:
# geometrical data
cmp.geom.scale = 61 * 0.0254 / 2
cmp.geom.inlet_hub_radius_ref = 0.3
cmp.geom.inlet_tip_radius = 1.
cmp.geom.exit_hub_radius = 0.4
cmp.geom.exit_tip_radius = 1.

In [ ]:
# calibration case
run = cmp.add_driver(NonLinearSolver('run'))

calib = run.add_child(RunSingleCase('calib'))
calib.design.extend(cmp.design_methods['calib'])

In [ ]:
cmp.fl_in.W = 330.
cmp.shaft_in.N = 5100
cmp.shaft_in.power = 10e6

cmp.run_drivers()

print('shaft power : ', cmp.shaft_in.power / 1e6, ' MW')

## design

In [ ]:
run = cmp.add_driver(NonLinearSolver('run'))

case = run.add_child(RunSingleCase('case'))
case.design.extend(cmp.design_methods['sizing'])

# addition conditions for isolated component
case.add_unknown('shaft_in.N').add_unknown('shaft_in.power')

In [ ]:
cmp.fl_in.W = 200.
cmp.run_drivers()

print('exit tip radius', cmp.geom.exit_tip_radius, ' m')
print('speed rotation ', cmp.shaft_in.N, ' rpm')

## partial physic only

In [ ]:
cmp = CompressorSimple('cmp', stage_count = 5, geom = False)
run = cmp.add_driver(NonLinearSolver('run'))
run.add_unknown('fl_in.W')
cmp.run_drivers()

In [ ]:
cmp = CompressorSimple('cmp', stage_count = 5, aero = False)
run = cmp.add_driver(NonLinearSolver('run'))
cmp.run_drivers()